In [9]:
#!/usr/bin/env python3

########################################################################
# File:randomizedMotifSearch.py
# Purpose: to find the promoter motif
#   stderr:
#   stdout:
#
# Author: Nicholas Chan
# History: 10/10/2021 Created
########################################################################

In [10]:
########################################################################
# CommandLine
########################################################################


class CommandLine():
    """
    Handle the command line, usage and help requests.

    CommandLine uses argparse,
    it implements a standard command line argument parser with various argument options,
    a standard usage and help, and an error termination exception Usage.
    """

    def __init__(self, inOpts = None):
        """
        CommandLine constructor.

        Implement a parser to interpret the command line argv string using argparse.
        """
        import argparse
        self.parser = argparse.ArgumentParser(
            description='Program prolog - a brief description of what this thing does',
            epilog='Program epilog - some other stuff you feel compelled to say',
            add_help=True,  # default is True
            prefix_chars='-',
            usage='%(prog)s [options] -option1[default] <input >output'
            )
        # Command line option to add iteration number as input. Default: 1000.
        self.parser.add_argument('-i', '--iterations', type=int, default=1000, action='store',
                                 help='number of times of iterations to find consensus motif')

        # Command line option to add consensus motif length as input. Default: 13.
        self.parser.add_argument('-k', '--motifLength', type=int, default=13, action='store',
                                 help='length of the target promoter motif')

        # Command line option to add number of pseudocounts as input. Default: 1.
        self.parser.add_argument('-p', '--pseudocount', type=float, default=1, action='store',
                                 help='number of pseudocounts')

        # Command line option to use Gibbs sampling to find the optimal consensus motif.
        self.parser.add_argument('-g', '--gibbsampling', type=float, default=1, action='store',
                                 help='implement Gibbs sampling')

        if inOpts is None:
            self.args = self.parser.parse_args()
        else:
            self.args = self.parser.parse_args(inOpts)

In [11]:

import sys

class FastAreader():
    """
    Read in files and preprocess.
    """
    def __init__(self, fname=''):
        """ Contructor: saves attribute fname. """
        self.fname = fname

    def doOpen(self):
        """ Open a file."""
        if self.fname == '':
            return sys.stdin
        else:
            return open(self.fname)

    def readFasta(self):
        """ Read in a fasta file and yield header and sequences separately"""
        header = ''
        sequence = ''

        with self.doOpen() as fileH:

            header = ''
            sequence = ''

            # skip to first fasta header
            line = fileH.readline()
            while not line.startswith('>'):
                line = fileH.readline()
            header = line[1:].rstrip()

            # Separate headers and sequences
            for line in fileH:
                if line.startswith('>'):
                    yield header, sequence
                    header = line[1:].rstrip()
                    sequence = ''
                else:
                    sequence += ''.join(line.rstrip().split()).upper()

        yield header, sequence

In [177]:
import sys
import os 
import numpy as np
from program import CommandLine
from fastaReader import FastAreader

# Handle iterations i in main function
class RandomMotifSearch():
    def __init__(self, k:'int', p:'', infile:'str', outfile:'str'):
        self.infile = infile
        self.outfile = outfile
        self.p = p
        self.k = k
        self.seqList = self.readData()[0]
        self.seqString = self.readData()[1]
        self.nullModel = self.nullMod()
        self.kmerPermutations = self.generateKMerPerms()
        
        self.motifList = self.motifify() # First motif list
        self.profileMatrix = self.profilify() # First motif profile matrix
#         self.motifList = self.reMotifify()
#         self.profileMatrix = self.profilify(self.motifList) # findProb dependent on this
        self.REscore = self.bestProfile()
        
    def readData(self) -> 'tuple(list,str)':
        '''
        Reads in fasta file as input. 
        Returns [0]: a list of seqs [1]: a string of catted seqs.
        '''
        seqList = []
        readData = FastAreader(self.infile).readFasta()
        for line in readData:
            seqList.append(line[1]) # Append seq and not head to seqList
        seqString = ''.join(seqList) # After all seq lines have been added to seqList, cat all into 1 str
        return (seqList,seqString)
    
    # Pseudo counts don't look like they'd make a big diff
    def nullMod(self) -> 'dict()':
        '''
        Returns null model with pseudocounts.
        '''
        nullProfDict = dict()
        countA = self.seqString.count('A')
        countC = self.seqString.count('C')
        countG = self.seqString.count('G')
        countT = self.seqString.count('T')
        countTot = countA + countC + countG + countT
        nullProfDict['A'] = (countA + self.p)/(countTot + self.p*4)
        nullProfDict['C'] = (countC + self.p)/(countTot + self.p*4)
        nullProfDict['G'] = (countG + self.p)/(countTot + self.p*4)
        nullProfDict['T'] = (countT + self.p)/(countTot + self.p*4)
        return nullProfDict
    
    def motifify(self) -> 'list': # Randomly generates motifs, independent
        ''' 
        Params: (self.seqList: 'list', self.k: 'int of motif length')
        Generates n number of sequences of length k (list of k-mers/motifs). 
        In other worrds, makes a list of randomly selected 
        k-mers/motifs from given list of seqs.
        '''
        motifs = []
        for seq in self.seqList:
            indexRange = np.random.randint(0,len(seq)-self.k)
            motifs.append(seq[indexRange:indexRange + self.k])
        return motifs
    
    # Counts check out, all columns add up to 1 and fractions match decimals
    def profilify(self) -> 'dict(lists)': # Independent
        ''' 
        Params: (motifList: 'list', p:'int'=0)
        Generates a profile matrix given a list of k-mers/motifs.
        Pseudocounts are added into the Experimental and NULL models
        in this method.
        '''
        # A-C-G-T correspond to 0-1-2-3
        matRowCount = len(self.motifList)
        motifLength = len(self.motifList[0])
        # Dictionary matrix where rows are bases [A,C,T,G] and cols are motif indices
        profileMatrix = dict(A=np.array([0]*motifLength), C=np.array([0]*motifLength), T=np.array([0]*motifLength), G=np.array([0]*motifLength))
        for motif in self.motifList: # Iterate first over motifs in motif list (input)
            idx=0 # Keep index of motif positions
            for base in motif: # Iterate over motif bases
                profileMatrix[base][idx]+=1
                idx+=1
        if self.p > 0: # If there are pseudocounts to be added
            for key in profileMatrix.keys():
                profileMatrix[key] = np.divide(profileMatrix[key] + self.p, matRowCount + (self.p*4)) # divides a list of motif base frequencies + pseudocount p by total bases per index + pseudocounts p*4
            return profileMatrix
        else: # If there are no pseudocounts to be added
            for key in profileMatrix.keys():
                profileMatrix[key] = np.divide(profileMatrix[key],matRowCount) # divides a list of motif base frequencies by total bases per index
            return profileMatrix  
        
    def findProb(self, kmer:'str') -> 'float':
        ''' 
        Params: (profileMatrix:'dict(lists)',kmer:'str')
        Finds the probability of a whole k-mer given a Profile.
        Find individual probabilities of each base in motif and finds product from each position.
        '''
        totProb = 1
        for idx in range(len(kmer)):
            totProb *= self.profileMatrix[kmer[idx]][idx]
        return totProb
    
    def generateKMerPerms(self): # independent
        ''' 
        Params: (seqList: 'List of raw sequences', k: 'int of k-mer length')
        Generates a list of lists of all k-mer permutations from raw sequences 
        '''
        kmerMat = []
        for seq in self.seqList:
            kmerList = []
            for idx in range(len(seq)-self.k):
                kmerList.append(seq[idx:idx+self.k])
            kmerMat.append(kmerList)
        return kmerMat
    
    def reMotifify(self) -> 'list of k-mers with the maximum probs':
        ''''
        Params: (kmerPermutations:'list of lists, profileMatrix:'4 x k dict of probabilities')
        Same type of output as motifify.
        Use this to generate motifs with maximum likelihood after initial random motif generation.
        A PERMUTATION MATRIX IS READ IN (list of lists)
        Each row contains all possible motif permutations per given sequence.
        '''
        maxPermList = []
        for permList in self.kmerPermutations:
            probList = []
            for perm in permList:
                probList.append(self.findProb(perm)) # Calculates probability of a motif given a profile and appends the probability
            maxPermList.append(permList[probList.index(max(probList))]) # Finds index of the motif with the highest probability in a PERMUTATION LIST and appends the permutation to maxPermList
        return maxPermList

    def findConsensus(self): # Can be called at end AFTER calling rms.bestProfile()
        '''
        Params: (profMat:'dict of np arrays')
        Finds a consensus sequence from the current profile matrix.
        '''
        motifLength = len(self.profileMatrix['A'])
        baseList = ['A','C','T','G']
        conSeq = ''
        for idx in range(motifLength):
            baseColList = []
            for base in baseList:
                baseColList.append((base,self.profileMatrix[base][idx]))
            # print(baseColList,end=' ')
            baseColList.sort(key=lambda x:x[1])
            # print(baseColList,end='\n\n')
            conSeq += baseColList[::-1][0][0] # Way of getting to value as baseColList looks like [['A',1],['B',2], ...,['F',6]]
        return conSeq
    
    def realEntropy(self, expModel:'dict(lists)'):
        '''
        Exp model is the profile matrix you want to score
        '''
        colLen = len(expModel['A'])
        bases = ['A', 'T', 'C', 'G']
        fullPart = []# list for sum of cols of RE eq
        for col in range(colLen): # calculation for RE for 1 column
            basePart = [] # list for inner sum of RE eq
            for base in bases: # calculation for RE for 1 base
                frac = expModel[base][col]/self.nullModel[base]
                #print(frac)
                logged = np.log2(frac)
                bpart = expModel[base][col]*logged
                basePart.append(bpart)
            fullPart.append(sum(basePart))
            #print(sum(basePart))
        return sum(fullPart)       
    
    def bestProfile(self):
#         matrix = self.motifList
#         profile = self.profileMatrix
#         reOld = 0
#         reNew = self.realEntropy(profile)
        RE = self.realEntropy(self.profileMatrix)
#         print("RE 1: ", end = ' ')
#         print(RE)
        diff = 0
        First = True
        while (diff > 0.0 or First):
            self.motifList = self.reMotifify()
            self.profileMatrix = self.profilify()
#             print(self.realEntropy(self.profileMatrix))
            First = False
            diff = self.realEntropy(self.profileMatrix) - RE
            RE = self.realEntropy(self.profileMatrix)
#             print("RE 2: ", end = ' ')
#             print(RE)
        return RE

## Shannon's Entropy
$H(x) = -\sum_{i = 1}^{n} P(x_{i}) log_{2} (P(x_{i}))$ 
<br/>
$H(x) = -\sum_{i \in [A,C,G,T]} P(x_{i}) log_{2} (P(x_{i}))$
- An encoding cost

## Relative Entropy
$D_{KL}(P||Q) = \sum_{i \in [A,C,G,T]} P(x_{i}) log_{2} \frac{P(x_{i})} {Q(x_{i})}$
- A distance between 2 profile probability distributions
- P and Q are 2 probability profile distributions
- We look at the log of a ratio of 2 probabilities (a comparison)
- We want to compare our observed profile to just the base composition of all our sequences
- Model Q is our NULL model
    - This is the probability of each of our 4 bases
- Model P describes our experimental model
    - This is the probability of each of our 4 bases in our selected motifs

In [185]:
aa =  RandomMotifSearch(k=13, p=1, infile="mdata/pogCrisprs",outfile='placeholder')
huh = [aa.seqList, aa.seqString, aa.nullModel, aa.motifList, aa.profileMatrix, aa.kmerPermutations]
print(huh[4])

{'A': array([0.66666667, 0.55555556, 0.55555556, 0.55555556, 0.44444444,
       0.11111111, 0.22222222, 0.66666667, 0.66666667, 0.66666667,
       0.66666667, 0.11111111, 0.55555556]), 'C': array([0.11111111, 0.22222222, 0.22222222, 0.22222222, 0.33333333,
       0.66666667, 0.55555556, 0.11111111, 0.11111111, 0.11111111,
       0.11111111, 0.33333333, 0.22222222]), 'T': array([0.11111111, 0.11111111, 0.11111111, 0.11111111, 0.11111111,
       0.11111111, 0.11111111, 0.11111111, 0.11111111, 0.11111111,
       0.11111111, 0.11111111, 0.11111111]), 'G': array([0.11111111, 0.11111111, 0.11111111, 0.11111111, 0.11111111,
       0.11111111, 0.11111111, 0.11111111, 0.11111111, 0.11111111,
       0.11111111, 0.44444444, 0.11111111])}


In [186]:
for i in range(1000):
    bb =  RandomMotifSearch(k=13, p=1, infile="mdata/p1860Crisprs",outfile='carbara')
    bb.bestProfile()
    print(bb.bestProfile(), bb.findConsensus())

4.215335217864263 CCCACAAAAACCA
3.5751926242143632 TAAAAACCCACAA
6.6222199241255355 ACTTAAAAACCCA
3.352422040572827 AAAAACCCACAAA
2.293648718415508 AAAACCCACAAAA
2.4410435931446828 AATAAAACCCCAC
5.785628256016556 AACTTAAAAACCC
3.5843964905643837 AAAACCCACAAAA
5.00974482778209 AAAAACTTAAAAA
3.486614091148714 AAAACTTAAAAAC
2.8513710821695035 AAAAACCCACAAA
6.6222199241255355 ACTTAAAAACCCA
5.599715386446567 AAACTTAAAAACC
2.8513710821695035 AAAAACCCACAAA
5.630953400085342 AGATGAAAACCTT
6.6222199241255355 ACTTAAAAACCCA
4.6336028782897865 AAAAACCCACAAA
6.141272658926792 ACGAAAAACTTAA
1.9012576323995694 CCAACAAACAACC
4.305618563082412 CACAAAAACCAAC
4.238107588116092 AAACCCGCAAAAA
3.5843964905643837 AAAACCCACAAAA
6.31225314622115 GACGAAAAACTTA
2.97673359433756 CCAAAACAAACCA
4.678025547195987 AACCCGCAAAAAC
3.5889046838118053 AACCCACAAAAAC
5.697696201879771 AAAACTTAAAAAC
6.6222199241255355 ACTTAAAAACCCA
7.565058447390234 GACGAAAAACTTA
2.888504372756013 ACAAAAACCAAAA
3.4654021955112304 AACCCACAAAA

5.057700917911026 ACCCGCAAAAACC
3.3674894961881643 AAAACCCACAAAA
4.575192624214364 TAAAAACCCACAA
6.6222199241255355 ACTTAAAAACCCA
4.65380578056855 AAACCCACAAAAA
4.183779229558187 AAAACTTAAAAAC
3.1454997264013747 ACAAAAACCAAAA
6.31225314622115 GACGAAAAACTTA
4.885035519038212 TACACAAGAACAA
4.238107588116092 AAACCCGCAAAAA
2.4720290160499228 AAAAACCAAAAAC
3.851371082169504 AAAACCCACAAAA
6.600147731881538 AAAACTTAAAAAC
3.601400437458806 AAAAACCAGCAAA
3.507759093140799 CCACAAAAAAACC
4.260439218931946 CACAAAAACCAAC
5.183646186929704 ACAGACGAAAAAC
6.6222199241255355 ACTTAAAAACCCA
3.91433016742171 CCCAAAAAACAAC
6.6222199241255355 ACTTAAAAACCCA
4.238107588116092 AAACCCGCAAAAA
3.772003431260776 AAACCCACAAAAA
2.7618549684599345 AAAAACCCACAAA
6.31225314622115 GACGAAAAACTTA
3.3007012215787244 AAAAACTTAAAAA
5.605733561579803 GAAAAACTTAAAA
3.0555222505635427 AAACCCACAAAAA
4.670823660443229 AAACTTAAAAACC
5.135605834368202 TTAAAAACCCACA
3.337251096780537 AAAAACCCACAAA
4.602075063612677 AAAACTTAAAAAC
3.8

4.238107588116092 AAACCCGCAAAAA
3.8641356715405752 AAAAGAGCAAAAA
3.3492494740898167 AAAAACCCACAAA
6.6222199241255355 ACTTAAAAACCCA
6.6222199241255355 ACTTAAAAACCCA
4.183779229558187 AAAACTTAAAAAC
5.8753589236867665 GAAAAACTTAAAA
6.60239633488464 TTAAAAACCCACA
3.3007012215787244 AAAAACTTAAAAA
5.535055628710849 CGAAAAACTTAAA
4.539944620203108 TAAAAACCCACAA
4.238107588116092 AAACCCGCAAAAA
6.6222199241255355 ACTTAAAAACCCA
3.3492494740898167 AAAAACCCACAAA
5.962918011779885 TAAAAACCCACAA
5.785628256016556 AACTTAAAAACCC
2.433075248115014 AAACCCACAAAAA
3.486614091148714 AAAACTTAAAAAC
3.352422040572827 AAAAACCCACAAA
2.433075248115014 AAACCCACAAAAA
2.8729932071949085 AACCCACAAAAAC
3.5843964905643837 AAAACCCACAAAA
5.402580425973322 ACTTAAAAACCCA
4.644277268361461 TAAAAACCCACAA
4.782189577872619 TAAAAACCCACAA
2.8187093449708596 AAAAAACACCCAA
5.697696201879771 AAAACTTAAAAAC
4.782189577872619 TAAAAACCCACAA
4.809085773868726 AAAACTTAAAAAC
3.4299012744054793 CAAAAACCAAAAA
6.6222199241255355 ACTTAAAAAC

3.3492494740898167 AAAAACCCACAAA
2.903524246598577 AAAACCCACAAAA
4.954044723599814 AAAAACTTAAAAA
3.224713669289791 CCAAAAAAAAACC
3.337251096780537 AAACCCACAAAAA
4.064868522107945 CCCACAAAAACCC
4.238107588116092 AAACCCGCAAAAA
4.65380578056855 AAAACCCACAAAA
3.599298625302661 AAAAACTTAAAAA
5.974848272979025 TTAAAAACCCACA
2.0329227154428673 AACAAAAAACAAA
4.137292889687704 AAAAACTTAAAAA
4.5672574007291304 AAACTTAAAAACC
3.532397272222951 CCACAAAAACCCC
6.141272658926793 CGAAAAACTTAAA
4.483164356162434 CCCACAAAAACCC
7.365871402405493 CTTAAAAACCCAC
4.46360389779405 ACCCACAAAAACC
4.535748889545325 AAAAACTTAAAAA
4.238107588116092 AAACCCGCAAAAA
7.365871402405493 CTTAAAAACCCAC
2.8513710821695035 AAAAACCCACAAA
6.48279339442603 AACTTAAAAACCC
3.3674894961881643 AAAACCCACAAAA
2.433075248115014 AAACCCACAAAAA
5.599715386446568 AAACTTAAAAACC
5.8753589236867665 GAAAAACTTAAAA
3.851371082169504 AAAACCCACAAAA
6.48279339442603 AACTTAAAAACCC
3.710344740371477 AAAACCCACAAAA
4.54369401099233 AAAAACTTAAAAA
5.79596

In [172]:
aa.reMotifify()

['ACTTAAAAACCCA',
 'ACGTAAAAAACCA',
 'ACTTAAAAACCCA',
 'ACTTAAAAACCCA',
 'ACTTAAAAACCCA']